In [35]:
openai_api_key = ""
gigachat_api_key = ""
data_path = 'data/RuBQ_2.0_dev.json'

# Устанавливаем необходимые зависимости

In [2]:
! pip install langchain
! pip install datasets
! pip install faiss-cpu
! pip install sentence-transformers


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 695.7 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz has inconsistent name: expected 'faiss-gpu', but metadata has 'faiss-cpu'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setu

In [ ]:
! pip install openai

In [ ]:
! pip install tiktoken

# Импортируем библиотеки

In [36]:
import time
import uuid
import requests
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
import pandas as pd
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts.example_selector import LengthBasedExampleSelector

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Пишем кастомный класс получения токена авторизации для GigaChat

In [37]:
class GigaChatSecureToken:
    access_token: str
    expires_at: int
    _offset: int = 60  # If token will be expired in {offset} seconds, is_expired() return true

    def __init__(self, access_token: str, expires_at: int):
        self.access_token = access_token
        self.expires_at = expires_at

    def is_expired(self):
        return round(time.time() * 1000) > self.expires_at + self._offset * 1000

# Определяем кастомный класс для работы с API GigaChat

In [38]:
class GigaChatLLM(LLM):
    api_key: str = None
    temperature: float = 0.7
    secure_token: GigaChatSecureToken = None

    @property
    def _llm_type(self) -> str:
        return "gigachat"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if not self.secure_token or self.secure_token.is_expired():
            print("Obtaining new secure token")
            self._auth()
            if not self.secure_token or self.secure_token.is_expired():
                # New token was not obtained
                print("ERROR: new token was not updated, cannot call LLM")
                return ""
        headers = {
            "Authorization": f"Bearer {self.secure_token.access_token}",
            "Content-Type": "application/json"
        }
        req = {
            "model": "GigaChat:latest",
            "messages": [{
                "role": "user",
                "content": prompt
            }],
            "temperature": self.temperature
        }
        response = requests.post("https://gigachat.devices.sberbank.ru/api/v1/chat/completions", headers=headers, json=req, verify=False)
        if response.status_code != 200:
            print(f"ERROR: LLM call failed, status code: {response.status_code}")
            return ""
        return response.json()["choices"][0]["message"]["content"]

    def _auth(self):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "RqUID": str(uuid.uuid4()),
            "Content-Type": "application/x-www-form-urlencoded"
        }

        scope_info = {"scope": "GIGACHAT_API_PERS"}
        response = requests.post("https://ngw.devices.sberbank.ru:9443/api/v2/oauth", data=scope_info, headers=headers, verify=False)
        if response.status_code != 200:
            print(f"ERROR: Something went wrong while obtaining secure token, status code: {response.status_code}")
            return
        content = response.json()

        expires_at = content["expires_at"]
        token = content["access_token"]
        if not (expires_at and token):
            print("ERROR: server returns empty values for fields 'expires_at' or 'access_token'")
            return
        self.secure_token = GigaChatSecureToken(token, expires_at)


llm = GigaChatLLM(api_key=gigachat_api_key)

In [39]:
llm(prompt="Пусть a = 5.  Сколько будет a в квадрате")

Obtaining new secure token


'Если мы возьмём число 5 и возведём его в квадрат, то получим 25.'

# Загружаем датасет

In [40]:
dataframe = pd.read_json(data_path)

In [41]:
dataframe

,uid,question_text,query,answer_text,question_uris,question_props,answers,paragraphs_uids,tags,RuBQ_version,question_eng
0,4,Какой стране принадлежит знаменитый остров Пасхи?,SELECT ?answer \nWHERE {\n wd:Q14452 wdt:P17 ?answer\n},Чили,[http://www.wikidata.org/entity/Q14452],[wdt:P17],"[{'type': 'uri', 'label': 'Чили', 'value': 'http://www.wikidata.org/entity/Q298', 'wd_names': {'ru': ['Республика Чили', 'Чили'], 'en': ['Chile', 'República de Chile', 'Republic of Chile', 'cl', 'Republica de Chile', 'CHI', '🇨🇱']}, 'wp_names': ['Чилийская']}]","{'with_answer': [10785, 10782, 10783], 'all_related': [10782, 10783, 10784, 10785, 10786, 53027, 53028, 53029, 53030, 53031, 53032, 53033, 53034, 53035, 53036, 35776, 35777, 35778, 35779, 35780, 51707, 51708, 51709, 51710, 51711]}",[1-hop],1,Which country does the famous Easter island belong to?
1,7,С какой музыкальной группой неразрывно связано имя Мика Джаггера?,SELECT ?answer \nWHERE {\n wd:Q128121 wdt:P361 ?answer \n},'Роллинг Стоунз',[http://www.wikidata.org/entity/Q128121],[wdt:P361],"[{'type': 'uri', 'label': 'The Rolling Stones', 'value': 'http://www.wikidata.org/entity/Q11036', 'wd_names': {'ru': ['The Rolling Stones', 'Роллинг Стоунс', 'Роллинг Стоунз'], 'en': ['The Rolling Stones', 'The Stones', 'Rolling Stones']}, 'wp_names': []}, {'type': 'uri', 'label': 'Джаггер/Ричардс', 'value': 'http://www.wikidata.org/entity/Q1818638', 'wd_names': {'ru': ['Джаггер/Ричардс'], 'en': ['Jagger-Richards', 'The Glimmer Twins', 'Jagger/Richards']}, 'wp_names': []}]","{'with_answer': [53040, 53037, 53038, 53039], 'all_related': [53037, 53038, 53039, 53040, 53041]}",[1-hop],1,Which music group is Mick Jagger's name inextricably linked to?
2,14,Где находится Летний сад?,SELECT ?answer \nWHERE {\n wd:Q1229234 wdt:P131 ?answer\n},Санкт-Петербург,[http://www.wikidata.org/entity/Q1229234],[wdt:P131],"[{'type': 'uri', 'label': 'Санкт-Петербург', 'value': 'http://www.wikidata.org/entity/Q656', 'wd_names': {'ru': ['Петроград', 'СПб', 'Петра творенье', 'Петербург', 'Культурная столица', 'С-Петербург', 'С.Петербург', 'Град Петров', 'Ленинград', 'Город на Неве', 'Питер', 'город Петра', 'Северная столица', 'Петрополь', 'Санкт-Петербург'], 'en': ['Petrograd', 'Petersburg', 'St Petersburg', 'Sankt-Peterburg', 'St. Petersburg', 'Saint Petersburg', 'Leningrad']}, 'wp_names': ['Ленинград']}]","{'with_answer': [25080, 25067], 'all_related': [53042, 53043, 53044, 53045, 53046, 53047, 53048, 53049, 53050, 53051, 53052, 53053, 53054, 53055, 53056, 53057, 53058, 53059, 53060, 53061, 25067, 25069, 25075, 25080, 25081]}",[1-hop],1,Where is the Summer garden?
3,22,Какой город является столицей Туркмении?,SELECT ?answer \nWHERE {\n wd:Q874 wdt:P36 ?answer\n},Ашхабад,[http://www.wikidata.org/entity/Q874],[wdt:P36],"[{'type': 'uri', 'label': 'Ашхабад', 'value': 'http://www.wikidata.org/entity/Q23438', 'wd_names': {'ru': ['Ашхабад', 'Полторацк', 'Асхабад', 'Ashgabat', 'Столица Туркмении', 'Ашгабат', 'Ашгабад'], 'en': ['Ishqabat', ""'Ishqabad"", 'Askabad', 'Ашхабад', 'Ashabad', 'Ishqabad', 'Ashgabat', 'Ashkabat', 'Ašgabat', 'Asgabat', 'Ak-Gaudan', 'Ashkabad', 'Aschchabad', 'Ashqabad', 'Ashgabad', 'Poltoratsk', 'Ashkhabad', 'Aşgabat', 'Askhabad', '`Ishqabad', '`Ishqábád']}, 'wp_names': []}]","{'with_answer': [14910], 'all_related': [11522, 14910, 14911, 14912, 14913, 14914, 53062, 53063, 53064, 53065, 53066, 53067, 53068, 16845, 53069, 16847, 53070, 53071, 16850, 16851, 16852, 11512, 11514, 11515, 11517]}",[1-hop],1,Which city is the capital of Turkmenistan?
4,25,"В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета ""Колокол""?",SELECT ?answer \nWHERE {\n wd:Q2533402 wdt:P159 ?answer\n},Лондон,[http://www.wikidata.org/entity/Q2533402],[wdt:P159],"[{'type': 'uri', 'label': 'Женева', 'value': 'http://www.wikidata.org/entity/Q71', 'wd_names': {'ru': ['Женева'], 'en': ['Genève', 'Geneve', 'Geneva', 'Genf', 'Geneva GE']}, 'wp_names': ['Женеве']}, {'type': 'uri', 'label': 'Лондон', 'v

In [42]:
dataframe = dataframe.dropna()
dataframe = dataframe.rename(columns = {'question_text': 'question', 'answer_text': 'answer'})[['question', 'answer']]

In [43]:
dataframe

,question,answer
0,Какой стране принадлежит знаменитый остров Пасхи?,Чили
1,С какой музыкальной группой неразрывно связано имя Мика Джаггера?,'Роллинг Стоунз'
2,Где находится Летний сад?,Санкт-Петербург
3,Какой город является столицей Туркмении?,Ашхабад
4,"В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета ""Колокол""?",Лондон
...,...,...
535,Какая дата релиза первого Сталкера?,20 марта 2007
536,Какая площадь Черного моря?,436 402 км²
537,Сколько серий в сериале Крепостная?,48
538,В каком году был основан Орден тамплиеров?,1119


# Трансформируем датафрейм в датафреймлоадер langchain. page_content_column в нашем случае поле question, так как именно над ним мы будем проводить векторизацию

In [44]:
loader = DataFrameLoader(dataframe, page_content_column = 'question')
data = loader.load()

In [45]:
data[:5]

[Document(page_content='Какой стране принадлежит знаменитый остров Пасхи?', metadata={'answer': 'Чили'}),
 Document(page_content='С какой музыкальной группой неразрывно связано имя Мика Джаггера?', metadata={'answer': "'Роллинг Стоунз'"}),
 Document(page_content='Где находится Летний сад?', metadata={'answer': 'Санкт-Петербург'}),
 Document(page_content='Какой город является столицей Туркмении?', metadata={'answer': 'Ашхабад'}),
 Document(page_content='В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета "Колокол"?', metadata={'answer': 'Лондон'})]

In [46]:
token_counts_max = dataframe['question'].str.split().str.len().max()

# Определяем модель для эмбеддингов.

In [47]:
# embedding_type = 'openai'
embedding_type = 'cointegrated/LaBSE-en-ru'

if embedding_type == 'openai':
    openai_batch_size = int(150000 / token_counts_max)
    embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key, chunk_size = openai_batch_size)
else:
    embeddings = HuggingFaceEmbeddings(model_name=embedding_type)

# Загружаем эмбеддинги в векторную бд

In [48]:
from langchain import FAISS
from faiss import IndexHNSWFlat
from time import sleep
from tqdm import tqdm

index_name = f'{data_path}_{embedding_type}'

index = IndexHNSWFlat()
faiss = FAISS(embeddings, index, None, None)

try:
    db = FAISS.load_local(index_name, embeddings)
except:
    db = None

if db is None:
    if embedding_type == 'openai':
        for batch_number in tqdm(range(0, len(data), openai_batch_size)):
            batch = data[batch_number: batch_number + openai_batch_size]
            batch_db = faiss.from_documents(batch, embeddings)
            if db is None:
                db = batch_db
            else:
                db.merge_from(batch_db)

            sleep(20)
    else:
        db = faiss.from_documents(data, embeddings)

    db.save_local(index_name)

In [49]:
query = "Кто изобрел телефон?"
db.similarity_search(query, k=5)

[Document(page_content='Кто был подлинным изобретателем телефона?', metadata={'answer': 'меуччи'}),
 Document(page_content='Кто изготавливает айфоны?', metadata={'answer': 'Китай'}),
 Document(page_content='В каком году был выпущен первый iphone?', metadata={'answer': '9 января 2007 года'}),
 Document(page_content='Кто основал компанию Форд?', metadata={'answer': 'Генри Форд'}),
 Document(page_content='Когда изобретен электрогенератор?', metadata={'answer': 'между 1853 и 1856 годами'})]

# Определяем ретривер - верхнеуровневую обертку над similarity_search, в которую захардкожены какие-то параметры (в нашем случае 5 ближайших соседей)

In [50]:
retriever = db.as_retriever(search_kwargs = {"k": 5, "score_threshold": 0.3})
retriever.get_relevant_documents(query)

[Document(page_content='Кто был подлинным изобретателем телефона?', metadata={'answer': 'меуччи'})]

# Определяем шаблонизатор промпта

In [51]:
prefix = """
Ответь на вопрос: {query}

У меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:
"""

context_example_template = """
Вопрос: {question}
Правильный ответ: {answer}
"""

suffix = ""

# Определяем пайплайн промпт template -> языковая модель

# Объединяем все в одну функцию

In [52]:
from typing import Tuple


def get_answer(question: str) -> Tuple[str, str]:
  res = retriever.get_relevant_documents(question)
  examples = [{"question": doc.page_content, "answer": doc.metadata["answer"]} for doc in res ]
  context_example_template_prompt = PromptTemplate(
      input_variables=["question", "answer"],
      template=context_example_template
  )

  example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=context_example_template_prompt,
    max_length=900
  )

  few_shot_prompt_template_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=context_example_template_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
  )

  chain = few_shot_prompt_template_prompt | llm

  answer = chain.invoke({ "query": question })


  prompt = few_shot_prompt_template_prompt.format(query = question)
  return answer, prompt

In [53]:
answer, prompt = get_answer("Кто был подлинным изобретателем телефона?")
print(prompt)
print(answer)


Ответь на вопрос: Кто был подлинным изобретателем телефона?

У меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:


Вопрос: Кто был подлинным изобретателем телефона?
Правильный ответ: меуччи

Извините, но я не могу согласиться с вашим ответом. Подлинным изобретателем телефона считается Александр Грэхэм Белл. Он разработал устройство, которое позволяло передавать голосовые сообщения на расстояние через провода. Это было в 1876 году.


# Сравним аугментированную модель с неаугментированной

In [54]:
questions = [
    "Кто был подлинным создателем первого телефона?",
    "Как называется столица США?",
    "Кто написал Войну и мир?",
    "Почему трава зеленая?",
    "Что такое водяной вор, клипсидра?"
]

In [55]:
compare_df = pd.DataFrame({"gigachat": [], "gigachat + augmentation": [], "prompt": []})
for question in tqdm(questions):
    llm_answer = llm(question)
    llm_augmented_answer, llm_augmented_prompt = get_answer(question)

    concat_df = pd.DataFrame.from_dict({"gigachat": [llm_answer], "gigachat + augmentation": [llm_augmented_answer], "prompt": [llm_augmented_prompt]})

    compare_df = pd.concat([compare_df, concat_df], ignore_index = True)
    if embedding_type == 'openai':
        sleep(20)

100%|██████████| 5/5 [00:15<00:00,  3.15s/it]


In [28]:
pd.set_option('display.max_colwidth', None)

In [56]:
compare_df

,gigachat,gigachat + augmentation,prompt
0,"Подлинным создателем первого телефона считается Александр Грэхэм Белл. Он работал над проектом вместе со своим помощником Томасом Уотсоном и получил патент на свое изобретение в 1876 году. Однако, стоит отметить, что идея создания устройства для передачи голоса на расстояние была известна задолго до Белла. Например, в Древнем Китае использовались воконы для передачи звуков на большие расстояния.",Вопрос: Кто был подлинным создателем первого телефона?\nПравильный ответ: Александр Грэхэм Белл,\nОтветь на вопрос: Кто был подлинным создателем первого телефона?\n\nУ меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:\n\n\nВопрос: Кто был подлинным изобретателем телефона?\nПравильный ответ: меуччи\n
1,Столицей Соединённых Штатов Америки является город Вашингтон.,Вашингтон,\nОтветь на вопрос: Как называется столица США?\n\nУ меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:\n
2,"""Война и мир"" был написан Львом Николаевичем Толстым.","Лев Толстой написал ""Войну и мир"".",\nОтветь на вопрос: Кто написал Войну и мир?\n\nУ меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:\n
3,"Зеленый цвет растениям придаёт хлорофилл — зеленый пигмент, который участвует в процессе фотосинтеза. Хлорофилл поглощает световую энергию и использует её для превращения углекислого газа и воды в глюкозу и кислород. Этот процесс происходит в хлоропластах растительных клеток.","Трава зеленая благодаря наличию хлорофилла в ее клетках. Хлорофилл поглощает световую энергию от солнца и использует ее для фотосинтеза — процесса, при котором растения превращают углекислый газ и воду в глюкозу и кислород. Зеленый цвет хлорофилла обусловлен его способностью поглощать световые волны определенной длины, что позволяет ему эффективно использовать энергию света для фотосинтеза.",\nОтветь на вопрос: Почему трава зеленая?\n\nУ меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:\n
4,"Водяной вор (или клипсидра) — это устройство для измерения уровня воды в резервуаре или другом контейнере. Он состоит из гибкой трубки, которая погружается в воду и имеет шкалу, на которой отображается уровень воды. Когда уровень воды достигает определенной отметки, водяной вор автоматически срабатывает и отключает подачу воды. Это очень полезное устройство для контроля уровня воды в различных системах, таких как системы полива, орошения и т.д.","Водяной вор (клипсидра) — это устройство для измерения уровня воды в резервуаре или цистерне. Он состоит из трубки с шкалой, которая погружается в воду, и датчика, который регистрирует уровень воды. Когда уровень воды достигает определенного значения, датчик срабатывает и подает сигнал о необходимости заполнения резервуара или цистерны.","\nОтветь на вопрос: Что такое водяной вор, клипсидра?\n\nУ меня есть несколько ответов на смежные вопросы. Если исходный вопрос и какой-то вопрос из списка очень близки по смыслу - используй представленный ответ из списка:\n"


In [58]:
compare_df.to_html('question-local-embedding.html')